In [1]:
import pandas as pd
import numpy as np
import xlrd as xl
import openpyxl as op
import datetime
import os

### Reading the file with losses, optimizing date format

In [3]:
folder = 'C:/Users/sgulbin/Work/Analysis/Insurance/Losses/'

df = pd.read_excel(folder+'Losses.xlsb'\
                    , sheet_name = 'sheet', engine = 'pyxlsb', parse_dates = True)
for i in ['Дата ДТП', 'Дата заявления в СК']:
    df[i] = pd.to_datetime(df[i], unit = 'd', origin = '1899-12-30')

# Adding count of damages to the df
grouped = df[['Номер убытка', 'ic']]
grouped = grouped.groupby('Номер убытка').count()
grouped['damages_count'] = grouped['ic']
grouped = grouped[['damages_count']]
df = pd.merge(df, grouped, left_on = 'Номер убытка', right_on = 'Номер убытка', how = 'left')

df['Index'] = df['Номер убытка'].map(str)+'-'+df['Сумма убытков'].map(str)
df['Номер убытка'] = df['Номер убытка'].astype('str')
df = df.sort_values('Номер убытка')
df = df.reset_index(drop = True)
df = df.add_suffix('_mf')

### Importing fresh losses, refreshing loss values, checking for duplicates

In [4]:
#Function to open the losses file, assign proper dtypes, add name of the insurance company, unique index and count of damages
def first_operations (ic, losses_file, loss_number_col, dmg_number_col, acc_date, reg_date, losses_sum_col):
    
    #Selecting data type of columns: loss_number as object and dates as datetime
    dtypes = {loss_number_col: object, dmg_number_col: object}
    dates_to_parse = [acc_date, reg_date]
    
    lf = pd.read_excel(folder+losses_file, sheet_name = 'sheet', dtype = dtypes, parse_dates = dates_to_parse)
    
    #Adding insurance company name
    lf['ic'] = ic

    #Calculating number of damages
    grouped = lf.groupby(loss_number_col).count()
    grouped['damages_count'] = grouped['ic']
    grouped = grouped[['damages_count']]
    
    #Adding number of damages to the losses file
    lf = pd.merge(lf, grouped, on = loss_number_col, how = 'left')
    lf = lf.sort_values(loss_number_col)
    lf = lf.reset_index(drop = True)
    
    #Adding unique index
    lf['Index'] = lf[loss_number_col].map(str)+'-'+lf[losses_sum_col].map(str)
    lf = lf.add_suffix('_lf')
    
    return lf

In [5]:
rgs = first_operations(ic = 'Росгосстрах',
                      losses_file = 'РГС_6_21.xlsx',
                      loss_number_col = 'Номер убытка',
                      dmg_number_col = '',
                      acc_date = 'Дата ДТП',
                      reg_date = 'Дата заявления в СК',
                      losses_sum_col = 'Сумма убытков')

In [8]:
alfa = first_operations(ic = 'АльфаСтрахование',
                      losses_file = 'Альфа_6_21.xlsx',
                      loss_number_col = 'Номер убытка',
                      dmg_number_col = 'Номер ущерба',
                      acc_date = 'Дата ДТП',
                      reg_date = 'Дата заявления в СК',
                      losses_sum_col = 'Сумма убытков')

In [9]:
ren = first_operations(ic = 'Ренессанс',
                      losses_file = 'Ренессанс_5_21.xlsx',
                      loss_number_col = 'Номер убытка',
                      dmg_number_col = '',
                      acc_date = 'Дата ДТП',
                      reg_date = 'Дата заявления в СК',
                      losses_sum_col = 'Сумма убытков')

In [6]:
#Function to update specific record
def update_record(df, col_old, col_new, losses_sum):
     df[col_old] = df[col_new].where((df[losses_sum+'_mf']!=df[losses_sum+'_lf'])&(df.Index_lf.notnull()), df[col_old])  

#Function to update the current losses file with new
def update_file (old, new, loss_num_col, losses_sum_col, cols):
    
    #Merging
    df = pd.merge(old, new, left_on = loss_num_col+'_mf', right_on = loss_num_col+'_lf', how = 'outer')
    #Replacing
    for i in cols:
        update_record(df = df, col_old = i+'_mf', col_new = i+'_lf', losses_sum = losses_sum_col)
    #Leaving only _mf columns
    mf = []
    for col in df.columns:
        if col[-3:] == '_mf':
            mf.append(col)
    df = df[mf]
    return df

In [7]:
test = update_file(old = df, new = rgs, loss_num_col = 'Номер убытка', losses_sum_col = 'Сумма убытков',\
                   cols = ['ic', 'Номер убытка', 'Дата ДТП', 'Дата заявления в СК', 'Сумма убытков'])

In [12]:
test = update_file(old = test, new = alfa, loss_num_col = 'Номер убытка', losses_sum_col = 'Сумма убытков',\
                   cols = ['ic', 'Номер убытка', 'Дата ДТП', 'Дата заявления в СК', 'Сумма убытков'])

In [14]:
test = update_file(old = test, new = ren, loss_num_col = 'Номер убытка', losses_sum_col = 'Сумма убытков',\
                   cols = ['ic', 'Номер убытка', 'Дата ДТП', 'Дата заявления в СК', 'Сумма убытков'])

In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16271 entries, 0 to 16270
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Index_mf                             16140 non-null  object        
 1   ic_mf                                16271 non-null  object        
 2   Номер убытка_mf                      16271 non-null  object        
 3   VIN_mf                               16135 non-null  object        
 4   vehicle_mf                           16140 non-null  object        
 5   brand_mf                             11601 non-null  object        
 6   brand (Archive)_mf                   13795 non-null  object        
 7   brand (List)_mf                      16140 non-null  object        
 8   Дата ДТП_mf                          16271 non-null  datetime64[ns]
 9   Дата страхового случая месяц-год_mf  16140 non-null  object        
 10  Дата заявл

In [16]:
test

,Index_mf,ic_mf,Номер убытка_mf,VIN_mf,vehicle_mf,brand_mf,brand (Archive)_mf,brand (List)_mf,Дата ДТП_mf,Дата страхового случая месяц-год_mf,Дата заявления в СК_mf,risk_mf,number of losses_mf,Исходящее ПВУ_mf,Сумма убытков_mf,Сумма убытков без исходящего ПВУ_mf,damages_count_mf
0,0017549473-28500.0,Росгосстрах,0017549473,Z94C251ABLR079587,Kia Rio X-Line,Делимобиль,Делимобиль,Делимобиль,2019-12-17 00:00:00,2019-12,2019-12-18,ОСАГО,1.0,0.0,28500.0,28500.00,1.0
1,0017550553-7500.0,Росгосстрах,0017550553,Z94C251ABLR079701,Kia Rio X-Line,Делимобиль,Делимобиль,Делимобиль,2019-12-04 00:00:00,2019-12,2019-12-18,ОСАГО,1.0,0.0,7500.0,7500.00,1.0
2,0017553684-26058.67,Росгосстрах,0017553684,Z94C251ABLR079487,Kia Rio X-Line,Делимобиль,Делимобиль,Делимобиль,2019-12-14 00:00:00,2019-12,2019-12-19,ОСАГО,1.0,0.0,26058.67,26058.67,1.0
3,0017567317-26500.0,Росгосстрах,0017567317,Z94C251ABLR079697,Kia Rio X-Line,Делимобиль,Делимобиль,Делимобиль,2019-12-25 00:00:00,2019-12,2019-12-30,ОСАГО,1.0,0.0,26500.0,26500.00,1.0
4,0017574433-7500.0,Росгосстрах,0017574433,Z94C251ABLR079605,Kia Rio X-Line,Делимобиль,Делимобиль,Делимобиль,2019-12-26 00:00:00,2019-12,2019-12-30,ОСАГО,1.0,0.0,7500.0,7500.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16266,NaN,Ренессанс,001GS21-021587,NaN,NaN,NaN,NaN,NaN,2021-01-12 02:35:00,NaN,2021-01-12,NaN,NaN,NaN,245084.0,NaN,NaN
16267,NaN,Ренессанс,001GS21-022935,NaN,NaN,NaN,NaN,NaN,2021-04-25 14:11:15,NaN,2021-05-05,NaN,NaN,NaN,280000.0,NaN,NaN
16268,NaN,Ренессанс,001GS21-022947,NaN,NaN,NaN,NaN,NaN,2021-04-25 14:11:15,NaN,2021-05-05,NaN,NaN,NaN,280000.0,NaN,NaN
16269,NaN,Ренессанс,001GS21-023420,NaN,NaN,NaN,NaN,NaN,2021-04-07 06:05:17,NaN,2021-05-06,NaN,NaN,NaN,102595.5,NaN,NaN


In [56]:
test.to_csv(folder + 'test.csv', index = False, encoding = 'utf-8-sig')

### Function for calculating sum and count of losses by insurance company and type of cover/risk

In [17]:
def lr_calc(file\
        , AP\
        , AP_column\
        , category\
        , period_start\
        , period_end\
        , ic_column\
        , risk_column\
        , losses_column\
        , losses_count\
        , loss_date_column\
        , report_date_column):
    if AP == True:
        file = file.loc[file[AP_column] == 'Anytime Prime']
        if category == 'by_loss_date':
            file = file.loc[(file[loss_date_column] >= period_start)&(file[loss_date_column] <= period_end)]
            file = file[[ic_column, risk_column, losses_column, losses_count]]
            ls = file.groupby([file[ic_column], file[risk_column]]).sum()
            ls = ls.unstack(level = 1)
            return ls
        else:
            file = file.loc[(file[report_date_column] >= period_start)&(file[report_date_column] <= period_end)]
            file = file[[ic_column, risk_column, losses_column, losses_count]]
            ls = file.groupby([file[ic_column], file[risk_column]]).sum()
            ls = ls.unstack(level = 1)
            return ls
    else:
        if category == 'by_loss_date':
            file = file.loc[(file[loss_date_column] >= period_start)&(file[loss_date_column] <= period_end)]
            file = file[[ic_column, risk_column, losses_column, losses_count]]
            ls = file.groupby([file[ic_column], file[risk_column]]).sum()
            ls = ls.unstack(level = 1)
            return ls
        else:
            file = file.loc[(file[report_date_column] >= period_start)&(file[report_date_column] <= period_end)]
            file = file[[ic_column, risk_column, losses_column, losses_count]]
            ls = file.groupby([file[ic_column], file[risk_column]]).sum()
            ls = ls.unstack(level = 1)
            return ls
        
ls = lr_calc(df\
        , False\
        , 'Бренд (Список)'\
        , 'by_loss_date'\
        , datetime.datetime(2020, 4, 1)\
        , datetime.datetime(2021, 3, 31)\
        , 'СК'\
        , 'Риск'\
        , 'Сумма убытков без исходящего ПВУ'\
        , 'Количество убытков'\
        , 'Дата страхового случая'\
        , 'Дата заявления убытка в СК')
print(ls)

KeyError: 'Дата страхового случая'

### Calculating exposure and earned premiums

In [122]:
osago = pd.read_excel('//delimobil.local/Delimobil/Risk/Страхование/Архив/Отчет по страхованию/OSAGO.xlsb'\
                    , sheet_name = 'OSAGO', engine = 'pyxlsb', parse_dates = True)
casco = pd.read_excel('//delimobil.local/Delimobil/Risk/Страхование/Архив/Отчет по страхованию/CASCO.xlsb'\
                    , sheet_name = 'CASCO', engine = 'pyxlsb', parse_dates = True)

period_start = datetime.datetime(2020, 4, 1)
period_end = datetime.datetime(2021, 3, 31)

# OSAGO
# for i in ['Дата начала срока страхования', 'Дата окончания срока страхования', 'Дата расторжения договора страхования']:
#     osago[i] = pd.to_datetime(osago[i], unit = 'd', origin = '1899-12-30')
# osago['period_start'] = period_start
# osago['period_end'] = period_end
# osago['0'] = 0

# osago['max'] = osago[['period_start', 'Дата начала срока страхования']].max(axis=1)
# osago['true_ending'] = osago[['Дата окончания срока страхования', 'Дата расторжения договора страхования']].min(axis=1)
# osago['min'] = osago[['period_end', 'true_ending']].min(axis=1)
# osago['diff'] = (osago['min'] - osago['max']).dt.days
# osago['exp'] = osago[['diff', '0']].max(axis=1)
# osago['exp'] = osago['exp']/364
# osago['EP'] = osago['exp']*osago['Подписанная премия по договору                      ']
# osago = osago[['СК', 'exp', 'EP']]
# osago = osago.groupby('СК').sum()
# osago['Риск'] = 'ОСАГО'
# osago['СК'] = osago.index
# osago = osago.set_index(['СК', 'Риск'])

# CASCO
for i in ['Дата начала срока страхования', 'Дата окончания срока страхования', 'Дата расторжения договора страхования']:
    casco[i] = pd.to_datetime(casco[i], unit = 'd', origin = '1899-12-30')
casco['period_start'] = period_start
casco['period_end'] = period_end
casco['0'] = 0

casco['max'] = casco[['period_start', 'Дата начала срока страхования']].max(axis=1)
casco['true_ending'] = casco[['Дата окончания срока страхования', 'Дата расторжения договора страхования']].min(axis=1)
casco['min'] = casco[['period_end', 'true_ending']].min(axis=1)
casco['diff'] = (casco['min'] - casco['max']).dt.days
casco['exp'] = casco[['diff', '0']].max(axis=1)
casco['exp'] = casco['exp']/364
casco['EP'] = casco['exp']*casco['WP в годовом выражении']
casco = casco[['СК', 'exp', 'EP']]
casco = casco.groupby('СК').sum()
casco['Риск'] = 'КАСКО'
casco['СК'] = casco.index
casco = casco.set_index(['СК', 'Риск'])

### Joining insurance losses and policies info

In [ ]:
ls = ls.join(casco, how='left')
ls = ls.join(osago, how = 'left')
ls = ls.join(casco, how = 'left')
ls = ls.join(osago, how = 'left')
ls = ls.unstack(level = 1)